# **ML Challenge 2025: Smart Product Pricing Solution**

#### Team Name: Algorithmic Minds
#### Team Members: Yash Handa, Suhani Kumari, Vishakha Gupta
#### Submission Date: 13/10/2025

In [1]:
import os
import re
import time
import urllib
from pathlib import Path
from functools import partial
from tqdm import tqdm
import numpy as np
import pandas as pd
from google.colab import files
from sentence_transformers import SentenceTransformer
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import models, transforms
from sklearn.model_selection import KFold

In [2]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
BATCH_SIZE = 64
NUM_EPOCHS = 100
LR = 1e-4
WEIGHT_DECAY = 1e-5
PATIENCE = 6
MODEL_SAVE_PATH = 'models/multimodal_model.pth'
SEED = 42
NUM_DOWNLOAD_WORKERS = 8
IMAGE_SIZE = (300, 300)
# Switch to all-mpnet-base-v2 for text embeddings
TEXT_EMB_MODEL_NAME = 'all-mpnet-base-v2'
ACCUMULATION_STEPS = 2 # gradient accumulation
NUM_FOLDS = 5

In [4]:
torch.manual_seed(SEED)
np.random.seed(SEED)
os.makedirs('models', exist_ok=True)
os.makedirs('features', exist_ok=True)

In [5]:
# upload train.csv from datasets folder
uploaded = files.upload()

Saving train.csv to train.csv


In [6]:
# upload test.csv from datasets folder
uploaded = files.upload()

Saving test.csv to test.csv


In [7]:
train_df = pd.read_csv('train.csv')

In [8]:
def clean_text(text):
    """Lightweight cleaning used before sending to sentence-transformers."""
    if isinstance(text, str):
        text = re.sub(r'<.*?>', ' ', text)
        text = re.sub(r'[^a-zA-Z0-9., ]', ' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text.lower()
    return ""

In [9]:
def smape(y_true, y_pred, eps=1e-8):
    num = np.abs(y_pred - y_true)
    den = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    return float(np.mean(num / (den + eps)) * 100.0)

In [10]:
def create_blank_image(save_path, size=IMAGE_SIZE):
    img = Image.new('RGB', size, (0, 0, 0))
    img.save(save_path)

In [11]:
def download_image(image_link, savefolder, max_retries=3, timeout=10):
    """Download single image with retries. Returns path or None."""
    try:
        if not isinstance(image_link, str) or image_link.strip() == "":
            return None
        filename = Path(image_link).name
        image_save_path = os.path.join(savefolder, filename)
        if os.path.exists(image_save_path):
            return image_save_path
        for attempt in range(max_retries):
            try:
                urllib.request.urlretrieve(image_link, image_save_path)
                return image_save_path
            except Exception as e:
                if attempt + 1 == max_retries:
                    # create placeholder
                    try:
                        create_blank_image(image_save_path)
                        return image_save_path
                    except Exception:
                        return None
                time.sleep(1 + attempt)
    except Exception:
        return None

In [12]:
def download_images_parallel(image_links, download_folder, workers=NUM_DOWNLOAD_WORKERS):
    os.makedirs(download_folder, exist_ok=True)
    results = []
    download_partial = partial(download_image, savefolder=download_folder)
    # simple threaded approach using map for portability
    from multiprocessing.pool import ThreadPool
    with ThreadPool(workers) as pool:
        for path in tqdm(pool.imap(download_partial, image_links), total=len(image_links)):
            results.append(path)
    return results

In [13]:
def compute_text_embeddings(df, text_col='catalog_content', out_pkl='features/train_text_embeddings.pkl', model_name=TEXT_EMB_MODEL_NAME):
    """
    Compute or load text embeddings.
    Output: pandas DataFrame with columns: sample_id, text_0, text_1, ..., text_{D-1}
    """
    if os.path.exists(out_pkl):
        print("Loading text embeddings from", out_pkl)
        return pd.read_pickle(out_pkl)

    if SentenceTransformer is None:
        raise RuntimeError("sentence-transformers not available. Install sentence-transformers.")

    print("Cleaning text...")
    df['clean_content'] = df[text_col].astype(str).apply(clean_text)

    print("Loading sentence-transformer:", model_name)
    model = SentenceTransformer(model_name, device=DEVICE)

    texts = df['clean_content'].tolist()
    print("Encoding texts:", len(texts))
    embeddings = model.encode(texts, batch_size=64, show_progress_bar=True, convert_to_numpy=True, device=DEVICE)
    print("Embeddings shape:", embeddings.shape)

    emb_df = pd.DataFrame(embeddings, columns=[f"text_{i}" for i in range(embeddings.shape[1])])
    emb_df.insert(0, 'sample_id', df['sample_id'].values)
    emb_df.to_pickle(out_pkl)
    print("Saved text embeddings to", out_pkl)
    return emb_df

In [14]:
def compute_image_features_from_folder(df, image_folder, out_pkl='features/train_image_features.pkl'):
    """
    Extract image features with EfficientNet-B3 pretrained (remove classifier).
    Output: pandas DataFrame with columns: sample_id, img_0, img_1, ..., img_{M-1}
    """
    if os.path.exists(out_pkl):
        print("Loading image features from", out_pkl)
        return pd.read_pickle(out_pkl)

    device = DEVICE
    preprocess = transforms.Compose([
        transforms.Resize(IMAGE_SIZE),
        transforms.CenterCrop(IMAGE_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

    try:
        eff = models.efficientnet_b3(weights=models.EfficientNet_B3_Weights.IMAGENET1K_V1)
    except Exception:
        # fallback: try efficientnet_b4
        eff = models.efficientnet_b4(weights=models.EfficientNet_B4_Weights.IMAGENET1K_V1)
    # remove classifier
    if hasattr(eff, 'classifier'):
        eff.classifier = nn.Identity()
    eff = eff.to(device)
    eff.eval()

    # determine feature dim by a dummy forward
    with torch.no_grad():
        dummy = torch.zeros((1, 3, IMAGE_SIZE[0], IMAGE_SIZE[1])).to(device)
        try:
            feat_dim = eff(dummy).cpu().numpy().flatten().shape[0]
        except Exception:
            # fallback: run a single real image shape through
            feat_dim = 1536

    features = []
    sample_ids = []
    print("Extracting image features for", len(df), "samples")
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        sample_id = row['sample_id']
        image_url = row.get('image_link', "")
        filename = os.path.basename(str(image_url))
        img_path = os.path.join(image_folder, filename)

        try:
            img = Image.open(img_path).convert('RGB')
            tensor = preprocess(img).unsqueeze(0).to(device)
            with torch.no_grad():
                feat = eff(tensor).cpu().numpy().flatten()
        except Exception as e:
            # fallback zero vector with expected dim
            # print(f"Warning: failed to extract image for sample {sample_id}. Using zeros. Err:", e)
            feat = np.zeros((feat_dim,), dtype=np.float32)

        features.append(feat)
        sample_ids.append(sample_id)

    feat_arr = np.vstack(features)
    img_df = pd.DataFrame(feat_arr, columns=[f"img_{i}" for i in range(feat_arr.shape[1])])
    img_df.insert(0, 'sample_id', sample_ids)
    img_df.to_pickle(out_pkl)
    print("Saved image features to", out_pkl)
    return img_df

In [15]:
def load_and_merge_features(text_pkl, img_pkl, csv_with_price=None):
    """
    Loads text_pkl and img_pkl (pandas), optionally merges price column from csv_with_price.
    Returns merged dataframe with columns: sample_id, text_*, img_*, price (if available)
    """
    text_df = pd.read_pickle(text_pkl)
    img_df = pd.read_pickle(img_pkl)
    merged = text_df.merge(img_df, on='sample_id', how='inner')

    if csv_with_price is not None:
        clean_df = pd.read_csv(csv_with_price)[['sample_id', 'price']]
        merged = merged.merge(clean_df, on='sample_id', how='inner')
    return merged

In [16]:
class TrainMultimodalDataset(Dataset):
    def __init__(self, df, target_col='price'):
        self.df = df.reset_index(drop=True)
        self.sample_ids = self.df['sample_id'].values
        self.y = self.df[target_col].values.astype(np.float32)
        # Drop identifier and target -> float32 features
        self.X = self.df.drop(columns=['sample_id', target_col]).values.astype(np.float32)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx]), torch.tensor(self.y[idx], dtype=torch.float32)

In [17]:
class TestMultimodalDataset(Dataset):
    def __init__(self, df):
        self.df = df.reset_index(drop=True)
        self.sample_ids = self.df['sample_id'].values
        self.X = self.df.drop(columns=['sample_id']).values.astype(np.float32)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx]), self.sample_ids[idx]

In [18]:
class MultimodalRegressor(nn.Module):
    def __init__(self, text_dim, img_dim, hidden_dim=256, dropout=0.3):
        super().__init__()
        self.text_dim = text_dim
        self.img_dim = img_dim

        # simple encoders
        self.text_fc = nn.Sequential(
            nn.Linear(self.text_dim, hidden_dim),
            nn.ReLU(inplace=True),
            nn.LayerNorm(hidden_dim),
            nn.Dropout(dropout)
        )
        self.img_fc = nn.Sequential(
            nn.Linear(self.img_dim, hidden_dim),
            nn.ReLU(inplace=True),
            nn.LayerNorm(hidden_dim),
            nn.Dropout(dropout)
        )

        # cross-attention / gating improvement: compute attention gate from concatenated features
        self.attn_gate = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim // 2),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_dim // 2, hidden_dim),
            nn.Sigmoid()
        )

        self.fusion = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(inplace=True),
            nn.LayerNorm(hidden_dim),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, 64),
            nn.ReLU(inplace=True),
            nn.LayerNorm(64),
            nn.Dropout(dropout),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        # x: [B, text_dim + img_dim]
        text = x[:, :self.text_dim]
        img = x[:, self.text_dim:self.text_dim + self.img_dim]
        t = self.text_fc(text)
        v = self.img_fc(img)
        # attention gate
        gate = self.attn_gate(torch.cat([t, v], dim=1))
        t = t * gate
        v = v * gate
        fused = torch.cat([t, v], dim=1)
        out = self.fusion(fused)
        return out.view(-1)

In [19]:
def train_loop(train_loader, val_loader, model, num_epochs=NUM_EPOCHS, accum_steps=ACCUMULATION_STEPS):
    # use Huber-like loss (Smooth L1) as requested
    criterion = nn.SmoothL1Loss()
    optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

    best_val_smape = float('inf')
    epochs_no_improve = 0

    scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=='cuda'))

    for epoch in range(1, num_epochs + 1):
        model.train()
        train_losses = []
        optimizer.zero_grad()
        for step, (X_batch, y_batch) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch} train", leave=False)):
            X_batch = X_batch.to(DEVICE)
            y_batch = y_batch.to(DEVICE)

            y_log = torch.log1p(y_batch)  # train on log target
            with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):
                preds_log = model(X_batch)
                loss = criterion(preds_log, y_log)
                loss = loss / accum_steps

            scaler.scale(loss).backward()

            if (step + 1) % accum_steps == 0 or (step + 1) == len(train_loader):
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), 3.0)
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()

            train_losses.append(loss.item() * accum_steps)

        avg_train_loss = float(np.mean(train_losses)) if train_losses else 0.0

        # evaluate
        val_preds, val_trues = evaluate(model, val_loader)
        val_preds = np.clip(val_preds, 0.0, None)
        val_smape = smape(val_trues, val_preds)

        print(f"Epoch {epoch} | TrainLoss(logHuber): {avg_train_loss:.6f} | Val SMAPE: {val_smape:.4f}%")
        scheduler.step(val_smape)

        if val_smape < best_val_smape - 1e-8:
            best_val_smape = val_smape
            epochs_no_improve = 0
            torch.save({
                'model_state_dict': model.state_dict(),
                'text_dim': model.text_dim,
                'img_dim': model.img_dim,
                'epoch': epoch,
                'val_smape': val_smape
            }, MODEL_SAVE_PATH)
            print(f"--> Best model saved (SMAPE {val_smape:.4f}%) at epoch {epoch}")
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= PATIENCE:
                print("Early stopping triggered.")
                break

    print(f"Training finished. Best val SMAPE: {best_val_smape:.4f}%")

In [20]:
def evaluate(model, loader):
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch = X_batch.to(DEVICE)
            y_batch = y_batch.to(DEVICE)
            out_log = model(X_batch)
            pred = torch.expm1(out_log).cpu().numpy()
            preds.append(pred)
            trues.append(y_batch.cpu().numpy())
    if len(preds) == 0:
        return np.array([]), np.array([])
    return np.concatenate(preds), np.concatenate(trues)

In [21]:
def predict_test_and_save(model_path, test_merged_df, out_csv='submission.csv'):
    ckpt = torch.load(model_path, map_location=DEVICE)
    text_dim = ckpt['text_dim']
    img_dim = ckpt['img_dim']
    model = MultimodalRegressor(text_dim=text_dim, img_dim=img_dim).to(DEVICE)
    model.load_state_dict(ckpt['model_state_dict'])
    model.eval()

    # ensure ordering of columns: sample_id then text_* then img_*
    cols = list(test_merged_df.columns)
    assert 'sample_id' in cols
    feature_cols = [c for c in cols if c != 'sample_id']

    test_ds = TestMultimodalDataset(test_merged_df[['sample_id'] + feature_cols])
    test_loader = DataLoader(test_ds, batch_size=256, shuffle=False, num_workers=2)

    sample_ids = []
    all_preds = []
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Predicting test"):
            X_batch, ids = batch
            X_batch = X_batch.to(DEVICE)
            out_log = model(X_batch)
            pred = torch.expm1(out_log).cpu().numpy()
            all_preds.extend(pred.tolist())
            sample_ids.extend(ids)

    preds = np.clip(np.array(all_preds), 0.0, None)
    out_df = pd.DataFrame({'sample_id': sample_ids, 'price': preds})
    # Ensure sample_id order matches test file order if provided
    out_df.to_csv(out_csv, index=False)
    print("Saved predictions to", out_csv)

In [22]:
if __name__ == "__main__":
    TRAIN_CSV = 'train.csv'
    TEST_CSV = 'test.csv'
    IMAGE_FOLDER_TRAIN = 'dataset/train_images'
    IMAGE_FOLDER_TEST = 'dataset/test_images'
    TEXT_PKL_TRAIN = 'features/train_text_embeddings.pkl'
    IMG_PKL_TRAIN = 'features/train_image_features.pkl'
    TEXT_PKL_TEST = 'features/test_text_embeddings.pkl'
    IMG_PKL_TEST = 'features/test_image_features.pkl'

    # --- Load CSVs ---
    train_df = pd.read_csv(TRAIN_CSV)
    test_df = pd.read_csv(TEST_CSV)
    print("Train samples:", len(train_df), "Test samples:", len(test_df))

    # --- Compute / Load text embeddings ---
    text_train_df = compute_text_embeddings(train_df, out_pkl=TEXT_PKL_TRAIN)
    text_test_df = compute_text_embeddings(test_df, out_pkl=TEXT_PKL_TEST)

    # --- Download images (train + test) to local folder ---
    train_image_links = train_df['image_link'].fillna("").astype(str).tolist()
    os.makedirs(IMAGE_FOLDER_TRAIN, exist_ok=True)
    print("Downloading train images...")
    download_images_parallel(train_image_links, IMAGE_FOLDER_TRAIN, workers=NUM_DOWNLOAD_WORKERS)

    test_image_links = test_df['image_link'].fillna("").astype(str).tolist()
    os.makedirs(IMAGE_FOLDER_TEST, exist_ok=True)
    print("Downloading test images...")
    download_images_parallel(test_image_links, IMAGE_FOLDER_TEST, workers=NUM_DOWNLOAD_WORKERS)

    # --- Compute image features (from downloaded folder) ---
    img_train_df = compute_image_features_from_folder(train_df, IMAGE_FOLDER_TRAIN, out_pkl=IMG_PKL_TRAIN)
    img_test_df = compute_image_features_from_folder(test_df, IMAGE_FOLDER_TEST, out_pkl=IMG_PKL_TEST)

    # --- Merge features with price (train) and without price (test) ---
    train_merged = text_train_df.merge(img_train_df, on='sample_id', how='inner').merge(train_df[['sample_id', 'price']], on='sample_id', how='inner')
    test_merged = text_test_df.merge(img_test_df, on='sample_id', how='inner')  # no price

    print("Train merged shape:", train_merged.shape)
    print("Test merged shape:", test_merged.shape)

    # --- K-Fold cross-validation ---
    feature_cols = [c for c in train_merged.columns if c not in ('sample_id', 'price')]
    text_dim = len([c for c in feature_cols if c.startswith('text_')])
    img_dim = len([c for c in feature_cols if c.startswith('img_')])
    print("Detected text_dim:", text_dim, "img_dim:", img_dim)

    kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=SEED)
    fold = 0
    val_smape_scores = []
    for train_idx, val_idx in kf.split(train_merged):
        fold += 1
        print(f"Starting fold {fold}/{NUM_FOLDS}")
        train_part = train_merged.iloc[train_idx].reset_index(drop=True)
        val_part = train_merged.iloc[val_idx].reset_index(drop=True)

        train_dataset = TrainMultimodalDataset(train_part)
        val_dataset = TrainMultimodalDataset(val_part)
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

        # build model for this fold
        model = MultimodalRegressor(text_dim=text_dim, img_dim=img_dim, hidden_dim=256, dropout=0.3).to(DEVICE)
        print(model)

        train_loop(train_loader, val_loader, model, num_epochs=NUM_EPOCHS, accum_steps=ACCUMULATION_STEPS)

        # after training load best saved model and evaluate on val
        if os.path.exists(MODEL_SAVE_PATH):
            ckpt = torch.load(MODEL_SAVE_PATH, map_location=DEVICE)
            model.load_state_dict(ckpt['model_state_dict'])
            val_preds, val_trues = evaluate(model, val_loader)
            val_preds = np.clip(val_preds, 0.0, None)
            val_smape = smape(val_trues, val_preds)
            val_smape_scores.append(val_smape)
            print(f"Fold {fold} val SMAPE: {val_smape:.4f}%")

    if len(val_smape_scores) > 0:
        print("K-Fold CV mean SMAPE:", np.mean(val_smape_scores))

    # --- predict on test and save CSV ---
    predict_test_and_save(MODEL_SAVE_PATH, test_merged, out_csv='submission.csv')

    print("All done. Submission: submission.csv")


Train samples: 75000 Test samples: 75000
Cleaning text...
Loading sentence-transformer: all-mpnet-base-v2


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

Epoch 41 | TrainLoss(logHuber): 0.080263 | Val SMAPE: 53.3170%


Epoch 42 | TrainLoss(logHuber): 0.077298 | Val SMAPE: 53.1633%


Epoch 43 | TrainLoss(logHuber): 0.074894 | Val SMAPE: 53.0818%
Early stopping triggered.
Training finished. Best val SMAPE: 53.0093%
Fold 5 val SMAPE: 53.0093%
K-Fold CV mean SMAPE: 52.76771545410156


Predicting test: 100%|██████████| 293/293 [00:04<00:00, 72.16it/s]


Saved predictions to submission.csv
All done. Submission: submission.csv


In [25]:
df_sub = pd.read_csv('submission.csv')
df_sub.shape

(75000, 2)

In [26]:
df_sub.head()

,sample_id,price
0,tensor(100179),12.246477
1,tensor(245611),23.463348
2,tensor(146263),22.017868
3,tensor(95658),2.920306
4,tensor(36806),16.240362


In [27]:
df_sub['sample_id'] = df_sub['sample_id'].astype(str).str.extract(r'tensor\((\d+)\)')[0].astype(int)
df_sub.to_csv('test_out.csv', index=False)

df_sub.head()

,sample_id,price
0,100179,12.246477
1,245611,23.463348
2,146263,22.017868
3,95658,2.920306
4,36806,16.240362
